In [1]:
import pandas as pd
import optuna
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

from sklearn.preprocessing import MaxAbsScaler

from sklearn.ensemble import RandomForestRegressor

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor


from utils import *


optuna.logging.set_verbosity(optuna.logging.WARNING)
import warnings
# Suppress pandas warnings
warnings.simplefilter(action='ignore', category=FutureWarning)



/home/khaled/scraper/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
versions = [
            'classic',
            'stations_basic', 
            'stations_weighted_all',
            'stations_weights_inverse',
            'stations_weights_gaussian',
            'stations_weights_logarithmic',
            'stations_weights_linear'
                ]

n_trials= 25
random_state = 42
test_size = 0.2

model_classes = [
    RandomForestRegressor,
    SVR,
    LinearRegression,
    Ridge,
    Lasso,
    ElasticNet,
    KNeighborsRegressor,
    DecisionTreeRegressor,
    
]

In [3]:
df_scores = pd.DataFrame(columns=['version', 'model', 'r2', 'mse', 'mae', 'best_params', 'validation_mae'])
studies = {}

In [4]:
for version in versions:
    for model_class in model_classes:
        print("="*50)
        print(f'{version} - {model_class.__name__}')
        X_train, X_test, y_train, y_test  = prepare_data(version=version, test_size=test_size, random_state=random_state)
        # Define the metric function for optimization
        metric_function = mean_absolute_error
        # Create the objective function with additional parameters
        objective = create_objective_function(X_train, y_train, metric_function, model_class)
        study = optuna.create_study(direction="minimize", study_name=f"{version}_{model_class.__name__}")
        
        # Optimize the study, use more trials for better performance
        if model_class == LinearRegression: # No hyperparameters to tune
            study.optimize(objective, n_trials=1, show_progress_bar=True,gc_after_trial=True) 
        else:      
            study.optimize(objective, n_trials=n_trials, show_progress_bar=True,gc_after_trial=True) 
            
        # train the model with the best parameters
        if model_class in [SVR, KNeighborsRegressor, LinearRegression]:
             model = model_class(**study.best_params)
        else:
            model = model_class(**study.best_params, random_state=random_state)
            
        scaler = MaxAbsScaler()
        # Fit the scaler on the training set and transform the training and testing sets
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
            
        model.fit(X_train_scaled, y_train)
        
        # evaluate the model
        y_pred = model.predict(X_test_scaled)
        r2 = r2_score(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        validation_mae = study.best_value
        new_row = {'version': version, 'model': model_class.__name__, 'r2': r2, 'mse': mse, 'mae': mae, 'best_params': study.best_params, 'validation_mae': validation_mae}
        df_scores = pd.concat([df_scores, pd.DataFrame([new_row])], ignore_index=True)
        # save the study
        studies[f'{version}_{model_class.__name__}'] = study

        

classic - RandomForestRegressor


  0%|          | 0/25 [00:00<?, ?it/s]

Best trial: 24. Best value: 124.819: 100%|██████████| 25/25 [09:03<00:00, 21.74s/it]


classic - SVR


Best trial: 23. Best value: 207.463: 100%|██████████| 25/25 [01:52<00:00,  4.49s/it]


...

Best trial: 16. Best value: 214.675: 100%|██████████| 25/25 [00:04<00:00,  5.69it/s]


stations_weights_linear - KNeighborsRegressor


Best trial: 23. Best value: 190.604: 100%|██████████| 25/25 [00:09<00:00,  2.61it/s]


stations_weights_linear - DecisionTreeRegressor


Best trial: 14. Best value: 162.245: 100%|██████████| 25/25 [00:05<00:00,  4.72it/s]


In [5]:
df_scores.sort_values(by='mae', ascending=True, inplace=False).head()


version                  model        r2           mse  \
8              stations_basic  RandomForestRegressor  0.768628  33361.879012   
0                     classic  RandomForestRegressor  0.770283  33123.169886   
24   stations_weights_inverse  RandomForestRegressor  0.766239  33706.345381   
32  stations_weights_gaussian  RandomForestRegressor  0.768286  33411.154501   
48    stations_weights_linear  RandomForestRegressor  0.762149  34295.984107   

           mae                                        best_params  \
8   108.915287  {'n_estimators': 1300, 'max_depth': 26, 'min_s...   
0   109.057792  {'n_estimators': 1500, 'max_depth': 28, 'min_s...   
24  111.119321  {'n_estimators': 900, 'max_depth': 28, 'min_sa...   
32  111.494079  {'n_estimators': 800, 'max_depth': 20, 'min_sa...   
48  111.923692  {'n_estimators': 500, 'max_depth': 18, 'min_sa...   

    validation_mae  
8       124.206365  
0       124.819479  
24      127.513741  
32      126.713682  
48      127.021905

In [6]:
# save studies object
add_name = '_'.join(versions)
import pickle
with open(f'studies/studies_classical.pkl', 'wb') as f:
    pickle.dump(studies, f)
# save results
df_scores.to_csv(f'results/results_classical.csv', index=False)

In [7]:
# load studies object

#studies = pickle.load(open(f'studies_{add_name}.pkl', 'rb'))